In [ ]:
# 1. 데이터 준비

In [ ]:
from tensorflow.keras.datasets import reuters
#matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# 기사 다운로드, 훈련용, 테스용으로 분리

# num_words는 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지만 사용할 것인지 조절
# 모든 단어를 사용할 것이므로 num_words=None
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

print('훈련용 뉴스 기사 : {}'.format(len(X_train)))
print('테스트용 뉴스 기사 : {}'.format(len(X_test)))

num_classes = max(y_train) + 1
print('카테고리 : {}'.format(num_classes))

In [ ]:
print(X_train[0]) # 첫번째 훈련용 뉴스 기사

In [ ]:
print(y_train[0]) # 첫번째 훈련용 뉴스 기사의 레이블

In [ ]:
# 8,982개의 훈련용 뉴스 기사의 길이가 대체적으로 어떤 크기를 가지는지 확인

print('뉴스 기사의 최대 길이 :{}'.format(max(len(l) for l in X_train)))
print('뉴스 기사의 평균 길이 :{}'.format(sum(map(len, X_train))/len(X_train)))

plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [ ]:
# 각 뉴스가 어떤 종류의 뉴스에 속하는지 기재되어있는 레이블 값의 분포 확인

fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(12,5)
sns.countplot(y_train)

In [ ]:
# 각 레이블에 대한 정확한 개수 확인

unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))
# label_cnt=dict(zip(unique_elements, counts_elements))
# 아래의 출력 결과가 보기 불편하여 병렬로 보고싶다면 위의 label_cnt를 출력

In [ ]:
# X_train 인덱스와 매치 단어 확인

word_to_index = reuters.get_word_index()
print(word_to_index)

In [ ]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [ ]:
print('빈도수 상위 28842번 단어 : {}'.format(index_to_word[28842]))

In [ ]:
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

In [ ]:
# 첫번째 훈련용 뉴스 기사인 X_train[0]가 어떤 단어들로 구성되어있는지를 복원

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

print(' '.join([index_to_word[index] for index in X_train[0]]))

In [ ]:
# LSTM으로 로이터 뉴스 분류하기

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# 등장 빈도 순서가 가장 많은 상위 1 ~ 1,000번째인 단어들만 분리

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

In [ ]:
# 패딩 : pad_sequences()를 사용하여 maxlen의 값으로 100

max_len = 100

X_train = pad_sequences(X_train, maxlen=max_len) # 훈련용 뉴스 기사 패딩
X_test = pad_sequences(X_test, maxlen=max_len) # 테스트용 뉴스 기사 패딩

In [ ]:
# 훈련용, 테스트용 뉴스 기사 데이터의 레이블에 원-핫 인코딩

y_train = to_categorical(y_train) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

In [ ]:
# 모델 설계

model = Sequential()
model.add(Embedding(1000, 120))  # (단어집합, 임베딩 벡터 차원)
model.add(LSTM(120))
model.add(Dense(46, activation='softmax'))  #46개의 카테고리 즉 46개의 뉴런

In [ ]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# 검증 데이터 손실이 4회 증가하면 학습 조기 종료

mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# 검증 데이터의 정확도가 이전보다 좋아질 경우에만 모델 저장

In [ ]:
# 모델 훈련

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
# categorical_crossentropy는 모델의 예측값과 실제값에 대해서 두 확률 분포 사이의 거리를 최소화하도록 훈련

history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_test, y_test))

In [ ]:
# 저장된 모델인 'best_model.h5'를 로드하고, 성능을 평가

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
# 에포크마다 변화하는 훈련 데이터와 검증 데이터(테스트 데이터)의 손실을 시각화

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show();